In [1]:
# import requered libraries

import os
import random
import uuid
import numpy as np
import matplotlib.pyplot as plt

import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
from tensorflow.keras.metrics import Precision, Recall

In [2]:
tf.__version__

'2.8.0'

In [3]:
# set up Gpu

# avoid dom errors by seting gpu memory consumption Growth

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

2022-03-29 21:06:36.014775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-29 21:06:36.047599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-29 21:06:36.047770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [9]:
#create data dir for training
work_dir ="/home/mostafa/Documents/Projects/FacialVerification"
DATA_DIR = "data"
POS_DIR = os.path.join(DATA_DIR,'positive') 
NEG_DIR = os.path.join(DATA_DIR,'negative')
ANK_DIR = os.path.join(DATA_DIR,'anchor')
!mkdir data/positive
!mkdir data/negative
!mkdir data/anchor

In [1]:
# download the Labeled Faces in the Wild data for negative labels form http://vis-www.cs.umass.edu/lfw/lfw.tgz
# unzip files
!tar -xf lfw.tgz


## collecting data 

In [11]:
# collecting negative data from labeled faces data and copy it to negative data dir
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join(work_dir,'lfw',directory)):
        current_path = os.path.join(work_dir,'lfw',directory,file)
        destination_path = os.path.join(work_dir,NEG_DIR,file)
        os.replace(current_path,destination_path)


#### collecting data from web cam for positive and anchor


In [8]:
# establish aconnection to webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    # reading frame
    ret, frame = cap.read()
    # cut frame to 250X250px
    frame=frame[120:120+250,200:200+250,:]
    # show the captured image
    if cv2.waitKey(1) & 0xFF == ord('a'):
        img_path = os.path.join(ANK_DIR,f"{uuid.uuid1()}.jpg")
        cv2.imwrite(img_path,frame)
    if cv2.waitKey(1) & 0xFF == ord('p'):
        img_path = os.path.join(POS_DIR,f"{uuid.uuid1()}.jpg") 
        cv2.imwrite(img_path,frame)
    cv2.imshow("Collected Image",frame)
    
    #breaking the system when done
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# release the webcam
cap.release()
# colse all image show
cv2.destroyAllWindows()

### Data Augmentation


In [3]:
def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img,max_delta=.02,seed=(np.random.randint(10),np.random.randint(10)))
        img = tf.image.stateless_random_contrast(img, lower=.8, upper=1,seed=(np.random.randint(10),np.random.randint(10)))
        img = tf.image.stateless_random_crop(img, size=(20,20,3),seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_flip_left_right(img,seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img,min_jpeg_quality=85, max_jpeg_quality=100,seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img,lower=.9, upper=1,seed=(np.random.randint(100),np.random.randint(100)))
        
        data.append(img)
    
    return data


In [8]:
img_path = os.listdir(ANK_DIR)[11]
img = cv2.imread(os.path.join(ANK_DIR,img_path))
data = data_aug(img)
for image in data:
    cv2.imwrite(os.path.join(ANK_DIR,f"{uuid.uuid1()}.jpg"), image.numpy())


2022-02-28 10:27:49.971096: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-28 10:27:49.971694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-28 10:27:49.971916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-28 10:27:49.972069: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [ ]:
# do augmantation to images and save it to it's dir
i=0
for img_path in os.listdir(POS_DIR):
    img = cv2.imread(os.path.join(POS_DIR,img_path))
    data = data_aug(img)
    for image in data:
        cv2.imwrite(os.path.join(POS_DIR,f"{uuid.uuid1()}.jpg"), image.numpy())
    

2022-03-29 21:09:05.519703: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-29 21:09:05.523830: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-29 21:09:05.524145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-29 21:09:05.524382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

## Loading data 

In [7]:
#data directories
anchor = tf.data.Dataset.list_files(ANK_DIR+'/*.jpg').take(4000)
positive = tf.data.Dataset.list_files(POS_DIR+'/*.jpg').take(4000)
negative = tf.data.Dataset.list_files(NEG_DIR+'/*.jpg').take(4000)

2022-02-28 10:29:55.867649: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-28 10:29:55.869722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-28 10:29:55.870356: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-28 10:29:55.870824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

### image preprocessing

In [8]:
def preprocess(file_path):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img)
    img = tf.image.resize(img,(100,100))
    img = img/255.0
    return img

In [10]:
positive = tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negative = tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positive.concatenate(negative)

In [11]:
data.as_numpy_iterator().next()

(b'data/anchor/a2c37b7a-93f4-11ec-883e-9cb6d067c58d.jpg',
 b'data/positive/699b6d1e-9870-11ec-b06a-9cb6d067c58d.jpg',
 1.0)

In [12]:
def preprocess_twin(input_img,validation_img,label):
    return (preprocess(input_img),preprocess(validation_img),label)

In [13]:
input_img,validation_img,label=preprocess_twin(*(data.as_numpy_iterator().next()))

In [17]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=12000)

In [19]:
# training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

                       

In [20]:
# test partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)
                       

# creating Model

In [21]:
def make_embedding(): 
    inp = Input(shape=(100,100,3),name="Input Image")
    C1 = Conv2D(64,(10,10),activation='relu')(inp)
    M1 = MaxPooling2D(64,(2,2),padding='same')(C1)
    
    C2 = Conv2D(128,(7,7),activation='relu')(M1)
    M2 = MaxPooling2D(64,(2,2),padding='same')(C2)
    
    C3 = Conv2D(128,(4,4),activation='relu')(M2)
    M3 = MaxPooling2D(64,(2,2),padding='same')(C3)
    
    C4 = Conv2D(256,(4,4),activation='relu')(M3)
    F1 = Flatten()(C4)
    FC1 = Dense(units=4069,activation='sigmoid')(F1)

    
    return Model(inputs=[inp], outputs=[FC1], name='embedding')

In [22]:
model = make_embedding()

In [23]:
model.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

### Build Distance Layer

In [24]:
class L1DIST(Layer):
    def __init__(self,**kwargs):
        super().__init__()
    
    def call(self,input_img,validation_img):
        return tf.math.abs(input_img-validation_img)

In [25]:
def make_siames_model():
    input_image = Input(shape=(100,100,3), name = "Input Embedding")
    validation_image = Input(shape=(100,100,3), name = "Validation Embedding")
    
    siames_layer = L1DIST()
    siames_layer._name ="DistanceLayer"
    distances = siames_layer(model(input_image),model(validation_image))
    
    classifier = Dense(1, activation = 'sigmoid')(distances)
    
    return Model(inputs = [input_image, validation_image], outputs = [classifier], name = 'SiameseNetwork')
    

In [26]:
siamese_model = make_siames_model()

In [27]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input Embedding (InputLayer)   [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 Validation Embedding (InputLay  [(None, 100, 100, 3  0          []                               
 er)                            )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4069)         38711589    ['Input Embedding[0][0]',        
                                                                  'Validation Embeddi

## Training Model 

### Loss Function 


In [28]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
optimizer = tf.optimizers.Adam(1e-4)


### Establish checkpoints

In [29]:
! mkdir training_checkpoints

mkdir: cannot create directory ‘training_checkpoints’: File exists


In [30]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')
checkpoint = tf.train.Checkpoint(opt=optimizer, siamese_model=siamese_model)

In [31]:
len(train_data.as_numpy_iterator().next())

3

In [32]:
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        #splet labels and data
        X = batch[:2]
        
        y = batch[2]
        
        #forward_propagation 
        
        yhat = siamese_model(X,training = True)
        loss = binary_cross_loss(y,yhat)
    
    #calculating Gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    optimizer.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
        
    return loss

In [33]:
def train(data,epochs):
    for epoch in range(1,epochs+1):
        
        print(f"\nEpochs {epoch}/{epochs}")
        progpar = tf.keras.utils.Progbar(len(data))
        r = Recall()
        p = Precision()
        for idx, batch in enumerate(data):
            loss=train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat)
            progpar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
            

In [34]:
# train and save model 
train(train_data,30)
siamese_model.save('siamesemodelv2.h5')



Epochs 1/30


2022-02-28 10:31:29.651156: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8301
2022-02-28 10:31:30.028037: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


350/350 [==============================] - 117s 330ms/step
0.06959162 0.9534384 0.9914339

Epochs 2/30
350/350 [==============================] - 114s 326ms/step
0.020433623 0.9978479 0.99820596

Epochs 3/30
350/350 [==============================] - 113s 324ms/step
0.0013737845 0.99318266 0.9974775

Epochs 4/30
350/350 [==============================] - 115s 328ms/step
0.0036669555 0.9917296 0.9956679

Epochs 5/30
350/350 [==============================] - 112s 321ms/step
0.0050732763 0.99821556 0.9996426

Epochs 6/30
350/350 [==============================] - 113s 324ms/step
0.050407737 0.9989354 0.99929005

Epochs 7/30
350/350 [==============================] - 114s 327ms/step
0.15560599 0.99785024 1.0

Epochs 8/30
350/350 [==============================] - 112s 319ms/step
0.023898352 0.9953538 0.99820787

Epochs 9/30
350/350 [==============================] - 113s 324ms/step
0.0007756312 0.9971367 0.9989244

Epochs 10/30
350/350 [==============================] - 114s 325ms/step
0.

In [35]:
test_input, test_validate, label = test_data.as_numpy_iterator().next()

In [38]:
y_hat = siamese_model.predict([test_input, test_validate])
y_hat

array([[3.9925942e-07],
       [3.9683215e-08],
       [4.1376462e-09],
       [9.8443907e-01],
       [5.9407168e-15],
       [5.0078039e-09],
       [6.9320882e-11],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999976e-01],
       [9.9999964e-01],
       [1.0000000e+00],
       [9.9999440e-01],
       [7.3846428e-07],
       [8.5329638e-10]], dtype=float32)

In [39]:
[1 if prediction > .5 else 0 for prediction in y_hat ]

[0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]

In [40]:
label

array([0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
      dtype=float32)

In [41]:
# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(label, y_hat)

# Return Recall Result
m.result().numpy()

1.0

In [42]:
# Creating a metric object 
m = Precision()

# Calculating the recall value 
m.update_state(label, y_hat)

# Return Recall Result
m.result().numpy()

1.0

In [43]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 

print(r.result().numpy(), p.result().numpy())

1.0 1.0


In [45]:
# loading Model
model = tf.keras.models.load_model('siamesemodelv2.h5',custom_objects={'L1DIST':L1DIST, "BinaryCrossentopy":tf.losses.BinaryCrossentropy()})

In [46]:
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input Embedding (InputLayer)   [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 Validation Embedding (InputLay  [(None, 100, 100, 3  0          []                               
 er)                            )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4069)         38711589    ['Input Embedding[0][0]',        
                                                                  'Validation Embeddi

## Real time test

### create dir for application data

In [37]:
!mkdir ./application_data
!mkdir ./application_data/input_image
!mkdir ./application_data/verification_images


#### verification Functions



In [47]:
def verify(model, detection_threshold, verification_threshold):
    
    results = []
    
    for image in os.listdir(os.path.join('./application_data','verification_images')):
        validation_img = preprocess(os.path.join('./application_data','verification_images',image))
        input_img = preprocess(os.path.join('./application_data','input_image',"input_image.jpg"))
        
        # make prediction 
        result = model.predict(list(np.expand_dims([input_img,validation_img],axis=1)))
        results.append(result)
    detection = np.sum(np.array(results)>detection_threshold)
    verification = detection / len(os.listdir(os.path.join('./application_data','verification_images')))
    verified = verification > verification_threshold
    return results, verified
        
        
    

In [ ]:
# establish aconnection to webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    # reading frame
    ret, frame = cap.read()
    # cut frame to 250X250px
    frame=frame[120:120+250,200:200+250,:]
    # show the captured image
    if cv2.waitKey(1) & 0xFF == ord('v'):
        img_path = os.path.join('./application_data','input_image',"input_image.jpg")
        cv2.imwrite(img_path,frame)
        results, verified = verify(model, .9, .7)
        print(f"verifid = {verified} \n")

        
    cv2.imshow("Collected Image",frame)
    
    #breaking the system when done
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# release the webcam
cap.release()
# colse all image show
cv2.destroyAllWindows()

verifid = False 

verifid = False 

